# Social Computing - Summer 2018
# Exercise 4 - Clustering

## Problem 4.1. K-means Clustering of social network data

Write a Python program that computes K-means clustering for a given social network dataset. The input dataset (file: networkinput.csv) contains anyonymized data from user profiles of a small social networking platform. Each line in the file represents one feature vector and is associated with a social network user. The vectors have a name ("userXYZ") and the following 4 features:
* Number of posts
* Number of comments
* Number of likes (on both posts and comments)
* Number of friends and followers (average)

By clustering the users, you identify users with similar activity patterns. This can be helpful for research but also for advertisers and polling firms.

Your program should compute K-means clustering for the dataset according to the formula discussed in the lecture ==> Minimizing the objective function:
$$\sum_{k=1}^{K} \sum_{\{n|x_n \in C_k \}} \|x_n - \mu_k\|^2$$
Such that:
$K$ is the number of clusters, $x_n$ is the nth point that belongs to the $k$th cluster, and $\mu_k$ is the centroid (prototype) of the $k$th cluster. (Refer to the lecture for details)
The K-means clustering algorithm should proceed as follows:
1. The program should start by parsing the dataset 
2. Assign four random centroids (prototypes).
3. Assign data points to the nearest centroid. 
4. Recompute the centroid values: The new centroid values of the kth centroid are calculated as the average values of the points currently in that centroid.
5. Repeat from point 3 till the values of the centroids don't change anymore.

<b>The output of your program should be a a list that asigns a cluster ID (0, 1, 2, 3) to every user in the input file. </b>
The first argument in that tuple should be the users's name (e.g., "user111") and the second argument should be the centroid id to which this user is associated to. e.g ('user111', 3).
**Note:** this output value should be the final centroid values that don't change anymore.


<b> After the clustering with k=4 is complete, run the code with the following centroid starting points:

centroids = {0: [9, 33, 29, 25], 1: [4, 44, 12, 41], 2: [10, 13, 44, 65], 3: [10, 44, 48, 70]}.

Have a look at the result and describe the common properties in each of the four groups</b> (max 5 sentences).


In [ ]:

### RUN THIS ON PYTHON 2.0 ##
import csv 
import random 
import numpy as np 
from scipy.spatial import distance 
import collections

# TODO: Read the network activity data set into a dictionary (observations_dictionary)
def read_data_set(dataset_path):
    observations = {}
    for line in open(dataset_path):
            row = line.strip().split(',')
            try:
                user_id, posts, comments, likes, followers = row[0], int(row[1]), int(row[2]), int(row[3]), int(row[4])
                observations[user_id] = posts, comments, likes, followers
            except KeyError:
                print("key error found! ")
    return observations



# TODO: Assign random centroids
# Hint: centroid values should be randomly assigned so that for each dimension of the centroid, the random values should fall
# between the maximum and the minimum value of that dimension of all the points in the data set.
# For example: for the dimension "Number of Comments", if the maximum value = 49, and the minimum value = 0, the value assigned to
# the dimension "Murder" of the centroid should be randomly assigned between 0 and 49

def create_random_centroid_values(observations, k): #, centroid):
    # find min and max of observations in each dimension
    # create dictionary with four random centroids within the observed space

    dimension_arr = np.array([])  # Initializing an empty numpy array
    for user, cluster_id in observations.items():
        if dimension_arr.size == 0:  ## when array is empty
            dimension_arr = np.asarray(list(cluster_id))  # First iteration, create the array
        else:
            dimension_arr = np.vstack((dimension_arr, cluster_id))

    minarr = np.amin(dimension_arr, axis=0)
    maxarr = np.amax(dimension_arr, axis=0)

    i = 0
    centroids = {}
    while (i < k):
        centroids[i] = [np.random.uniform(minarr[0], maxarr[0]), np.random.uniform(minarr[1], maxarr[1]),
                        np.random.uniform(minarr[2], maxarr[2]), np.random.uniform(minarr[3], maxarr[3])]
        i += 1

    return centroids
    
    
# Assign centroid ID for each of the data points. Each data item is assigned to its closest centroid
def update_observation_centroids(observations, centroids, k, observation_centroids=None):
    # create dictionary mapping each observation to a centroid index
    if centroids == None:
        # # initial run: For each centroid i of the k centroids, create random values
        centroids = create_random_centroid_values(observations, k)
    # Otherwise
    observation_centroids = observations.copy()
    centroid_id = 5
    ## TODO: might have to change
    mindist = float("inf")
    for key in observations:
          # TODO: Each centroid i of the K centroids is the average of the data values previously assigned to that centroid i
        for c in centroids:
            if distance.euclidean(observations[key], centroids[c]) < mindist:
                mindist = distance.euclidean(observations[key], centroids[c])
                centroid_id = c
        observation_centroids[key] = centroid_id
    # print("Observation Centroids: \n",observation_centroids)
    return observation_centroids # TODO: Return the newly created observation centroids


# Create new centroid values for each cluster as the mean of data values for the points in that cluster
def update_centroid_values(observations, observation_centroids, k):
    updated_centroids = {}
    posts = [[0],[0],[0],[0]]
    comments = [[0],[0],[0],[0]]
    likes = [[0],[0],[0],[0]]
    followers = [[0],[0],[0],[0]]
    for user in observations:
        i = observation_centroids[user]
        posts[i].append(observations[user][0])
        comments[i].append(observations[user][1])
        likes[i].append(observations[user][2])
        followers[i].append(observations[user][3])

    ## Converting list of list to np.array of array so that we can perform np.mean operation
    posts = np.array([np.array(xi) for xi in posts])
    comments = np.array([np.array(xi) for xi in comments])
    likes = np.array([np.array(xi) for xi in likes])
    followers = np.array([np.array(xi) for xi in followers])

    ## since all arrays of arrays are of different lengths hence specifying the axis in np.mean didn't work
    posts_mean = [np.mean(i) for i in posts]
    comments_mean = [np.mean(i) for i in comments]
    likes_mean = [np.mean(i) for i in likes]
    followers_mean = [np.mean(i) for i in followers]

    for i in range(0,k):
        updated_centroids[i] = [posts_mean[i], comments_mean[i], likes_mean[i], followers_mean[i]]
#     print("Updated Centroids: \n",updated_centroids)

    return updated_centroids    

def stopping_criteria(centroids, new_centroids):
    if centroids == None or new_centroids == None:
        return False
    # Extract the values and store them in lists
    centroid_values = centroids.values()
    new_centroid_values = new_centroids.values()

    # Compare the 2 lists to see if they contain the same values
    equal = True
    for i in centroid_values:
        equal = equal & (i in new_centroid_values)

    return equal

def  calculate_k_means_clustering(data_set_path, k):
    observations = read_data_set(data_set_path)
    centroids = {0: [9, 33, 29, 25], 1: [4, 44, 12, 41], 2: [10, 13, 44, 65], 3: [10, 44, 48, 70]}
    new_centroids = dict.fromkeys(range(0,k),0)
    count = 0
    # TODO: Compare the new centroid dictionary with the old centroid dictionary

    while (1):
        centroids = new_centroids
        # Go through all observations and assign them to the centroid closest to them.
        observation_centroids = update_observation_centroids(observations, centroids, k)
        # Compute updated centroids by averaging the values of the observations assigned to the centroid in question,
        # using the dictionary calculated in the update_observation_centroids method.
        new_centroids = update_centroid_values(observations, observation_centroids, k)
#         print('New centroids:', new_centroids)
#         print("Old centroids:", centroids)
        count += 1
        if stopping_criteria(centroids, new_centroids) == True:
            break
    print("Attempt:= ", count)
    return observation_centroids

  
        
        
# # run code
# calculate_k_means_clustering('networkinput.csv', 4)
dict_centroids = calculate_k_means_clustering('networkinput.csv', 4)
    # Sorting the dictionary
dict_final = collections.OrderedDict(sorted(dict_centroids.items(), key=lambda x: x[1]))

print("\n \n")
print(dict_final)    


<h2>Remark on Numerics of K-Means (also for GMMs)</h2>

When using a random seed, it might happen that a cluster is finally empty. A common strategy to prevent such results is a (random) re-initialization of empty clusters. In order to get everyone on the same boat, we have also provided a given seed ([9, 33, 29, 25], [4, 44, 12, 41], [10, 13, 44, 65], [10, 44, 48, 70]). That means, everyone should be able to produce the exact same results.

With this seed, you should not get any empty clusters any empty final clusters.

If you want to compare with library functions, you should get the same results. Since there is a huge discussion going on, and multiple code snippets have been posted, here is a very simple code using sklearn.cluster:

<code>
from sklearn.cluster import KMeans
from numpy import genfromtxt

observations = genfromtxt('networkinput.csv', delimiter=',')[:, 1 : 5]
centers = np.asarray([[9, 33, 29, 25], [4, 44, 12, 41], [10, 13, 44, 65], [10, 44, 48, 70]])
kmeans = KMeans(n_clusters=4, init=centers).fit(observations)
result = kmeans.predict(observations)
print sum(result == 0), sum(result == 1), sum(result == 2), sum(result == 3)
</code>


This does not produce any final empty clusters and it can be used as a reference point for validating your implementation.

While this code does not produce any final empty clusters, sklean.cluster.KMeans obviously re-initializes an empty cluster. As already mentioned by multiple participants, an empty cluster will be created in iteration 1.

One common way to fix this situation is the re-initialization of that particular cluster using a random point or points that are far away from their centroid. Setting an empty cluster to zero is usually not a good idea - it also depends on the specific underlying space, e.g. the range of our observations. For this particular exercise, setting the center to zero creates a similar effect as setting the center to points that are far away from their centroid, because 0 is located at the boundaries of our data points.
 
You can check out the method used by sklean.cluster.KMeans here: https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/cluster/_k_means.pyx.

Throwing away an empty cluster is usually not a good idea, since we want to specify the amount of clusters in advance. A 4-means-clustering is supposed to create 4 final prototypes.

## Optional Problem 4.2. Girvan-Newman Algorithm 

This problem is optional. That means you do not HAVE to do it to get full marks. However, it is certainly beneficial to counterbalance potential flaws of your solution to problem 4.1. It is, of course, also beneficial if you want to learn something. 

The Girvan-Newman algorithm is an efficient algorithm for computing graph clustering.

Write a Python program that computes the best clustering for the Krackhardt Kite graph.
Remember from the lecture that the central idea of the Girvan-Newman algorithm is to compute the edge betweenness in the input graph. Large betweenness value is an indication that the corresponding edge is a bridge between two clusters in the graph, and cutting that edge means isolating those clusters.
The algorithm proceeds by determining edge betweenness values for all the edges in the graph, and removing the edge with the highest betweenness value and repeating until there are no more edges.
The output of the Girvan-Newman algorithm is a dendrogram of clusters where individual vertices are at the bottom. Therefore, it's necessary to cut that dendrogram and determine the best cluster. Best clustering is the one with the highest graph modularity value, which is determined by the formula:
$$ Q = \sum_{i} (e_{ii} - a_i^2) $$
where $e_{ii}$ sums the fraction of graph edges that connects nodes in the ith cluster. And $a_i$ is the fraction of edges that connect to the ith cluster (see lecture for details)
The input to the Python program will be the Krackhardt Kite Graph as an igraph Graph object. The output should be a tuple of two arguments:
The first should be the value of the best modularity corresponding to the best graph clustering (a floating point number). The second argument should be a tuple of igraph Graph objects representing the clusters. 

For computing the edge betweenness, you can use any APSP-algorithm (e.g. Floyd Warshall) or (better) you can use a variation of the algorithm for calculating hortest path betweenness centrality by Ulrik Brandes (U. Brandes, A faster algorithm for betweenness centrality. Journal of Mathematical Sociology 25, 163–177 (2001), that was referenced in the paper M. E. J. Newman and M. Girvan: Finding and evaluating community structure in networks. Arxiv, 2003) and that is state of the art for computing shortest path betweenness centrality.
You may in any case use a library procedure for calculating the edge betweenness centrality. 

In [ ]:
import igraph
import Queue
from __future__ import division
import numpy as np


def calculate_node_degrees(g):
    # HINT: You will need to calculate the adjacency matrix of the graph
    # Some Graph methods from igraph could be helpful

def calculate_modularity(g, original_deg_dict, m):
    modularity = 0
    degree_dict = calculate_node_degrees(g)
    connected_components = g.components()
    for i in range(len(connected_components)):
        subgraph = connected_components.subgraph(i) # each subgraph represents a cluster in the graph
        e = 0 # Fraction of edges that connect to cluster
        a = 0 # Fraction of edges that connect to cluster with random connections between edges
        for v in subgraph.vs:
            e += degree_dict[v.index]
            a += original_deg_dict[v.index]
        # TODO: Calculate the modularity
    return modularity
        
# TODO: Calculate edge betweeness  
def calculate_edge_betweenness(g):

def calculate_girvan_newman_clustering(g):
    m = # TODO: The original number of edges
    original_degree = calculate_node_degrees(g)
    while # TODO: Graph still has edges:
        Q = calculate_modularity(g, original_degree, m)
        if (Q > largest_modularity):
            largest_modulatirty = Q
            edge_betweenness = calculate_edge_betweenness(g) # TODO: get edge with the highest betweenness value
            # TODO: delete the edge with the maximum betweenness value
            # Hint: Some built-in methods from igraph could be helpful
    
    # Finally
    return final_graph_clustering, largest_modularity

# Program's entry point:
g = igraph.Graph.Famous('Krackhardt_Kite') # Connected, Unweighted, undirected social network
calculate_girvan_newman_clustering(g)